In [1]:
#!/usr/bin/python
import talib
import requests
import python_bitbankcc
import pandas as pd
import numpy as np
from datetime import datetime,date, timedelta
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter

In [2]:
MEAN_RANGE = 5
K = 1.6
start_day = "20190101"
start_dt = datetime.strptime(start_day, "%Y%m%d")
end_day = "20200730"
end_dt = datetime.strptime(end_day, "%Y%m%d")
dt = start_dt
print(dt)

2019-01-01 00:00:00


In [3]:
%time

API = python_bitbankcc.public()
pair = 'btc_jpy'
period = '1hour' # 1min, 5min, 15min, 30min, 1hour, 4hour, 8hour, 12hour, 1day, 1week
ohlcv = []
while dt <= end_dt:
    ohlcv.extend(API.get_candlestick(pair, period, datetime.strftime(dt, "%Y%m%d"))['candlestick'][0]['ohlcv'])
    dt = dt + timedelta(days=1)
df = pd.DataFrame(ohlcv, columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Time'])
df['Time'] = [datetime.fromtimestamp(float(time)/1000) for time in df['Time']]
df.drop('Volume', axis=1, inplace=True)
df.set_index('Time', inplace=True)
df.tail()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 4.29 µs


,Open,High,Low,Close
Time,,,,
2020-07-30 05:00:00,1162981,1164800,1157478,1160915
2020-07-30 06:00:00,1161079,1164000,1155000,1161600
2020-07-30 07:00:00,1161600,1161701,1150213,1154398
2020-07-30 08:00:00,1154791,1161318,1154093,1161200
2020-07-30 09:00:00,1161282,1161475,1158123,1158123


In [5]:
df['Open'] = df['Open'].astype(np.int64)
df['High'] = df['High'].astype(np.int64)
df['Low'] = df['Low'].astype(np.int64)
df['Close'] = df['Close'].astype(np.int64)

In [6]:
df1 = df.copy()
df1['diff'] = df1['High'] - df1['Low']
df1['range'] = df1['diff'].rolling(MEAN_RANGE).sum().fillna(df1['diff'].rolling(MEAN_RANGE).sum().mean())
df1['range_mean'] = df1['range']/MEAN_RANGE
df1.loc[df1['range_mean']*K+df1['Open'] < df1['High'], 'signal'] = 'buy'
df1.loc[df1['Open']-df1['range_mean']*K > df1['Low'], 'signal'] = 'sell'
print(df1['signal'].value_counts(ascending=True))
print('DataFrame shape: {}'.format(df1.shape))
df1.tail()

sell    593
buy     604
Name: signal, dtype: int64
DataFrame shape: (13834, 8)


,Open,High,Low,Close,diff,range,range_mean,signal
Time,,,,,,,,
2020-07-30 05:00:00,1162981,1164800,1157478,1160915,7322,42981.0,8596.2,NaN
2020-07-30 06:00:00,1161079,1164000,1155000,1161600,9000,46286.0,9257.2,NaN
2020-07-30 07:00:00,1161600,1161701,1150213,1154398,11488,44464.0,8892.8,NaN
2020-07-30 08:00:00,1154791,1161318,1154093,1161200,7225,42530.0,8506.0,NaN
2020-07-30 09:00:00,1161282,1161475,1158123,1158123,3352,38387.0,7677.4,NaN


In [7]:
from backtesting import Strategy
from backtesting.lib import crossover

def EMA_Backtesting(values, n):
    """
    Return exponential moving average of `values`, at
    each step taking into account `n` previous values.
    """
    close = pd.Series(values)
    return talib.EMA(close, timeperiod=n)

class EmaCrossStrategy(Strategy):
    
    # Define the two EMA lags as *class variables*
    # for later optimization
    n1 = 5
    n2 = 10
    
    def init(self):
        # Precompute two moving averages
        self.ema1 = self.I(EMA_Backtesting, self.data.Close, self.n1)
        self.ema2 = self.I(EMA_Backtesting, self.data.Close, self.n2)
    
    def next(self):       
        # If ema1 crosses above ema2, buy the asset
        if crossover(self.ema1, self.ema2):
            self.position.close()
            self.buy()

        # Else, if ema1 crosses below ema2, sell it
        elif crossover(self.ema2, self.ema1):
            self.position.close()
            self.sell()

/opt/conda/lib/python3.7/site-packages/backtesting/_plotting.py:44: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [8]:
from backtesting import Backtest
# start from 1 BTC with the commission 0.12%

df_ = df.copy()
bt_ema = Backtest(df_, EmaCrossStrategy, cash=1, commission=.0012, trade_on_close=True)
bt_ema.run()

Start                     2019-01-01 00:00:00
End                       2020-07-30 09:00:00
Duration                    576 days 09:00:00
Exposure [%]                          99.8265
Equity Final [$]                     0.433788
Equity Peak [$]                       2.06154
Return [%]                           -56.6212
Buy & Hold Return [%]                 184.485
Max. Drawdown [%]                    -80.9119
Avg. Drawdown [%]                    -5.64773
Max. Drawdown Duration      391 days 03:00:00
Avg. Drawdown Duration       12 days 19:00:00
# Trades                                 1208
Win Rate [%]                          23.4272
Best Trade [%]                        23.9434
Worst Trade [%]                      -11.6215
Avg. Trade [%]                     -0.0397636
Max. Trade Duration           3 days 15:00:00
Avg. Trade Duration           0 days 12:00:00
Expectancy [%]                        1.30735
SQN                                 -0.634194
Sharpe Ratio                      

In [9]:
%time

n1=list(range(5, 20))
n2=list(range(15, 30))
stats = bt_ema.optimize(n1=n1, n2=n2,
                    maximize='Equity Final [$]', constraint=lambda p: p.n1 < p.n2)
print(stats)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.01 µs


Start                               2019-01-01 00:00:00
End                                 2020-07-30 09:00:00
Duration                              576 days 09:00:00
Exposure [%]                                    98.6554
Equity Final [$]                                3.16438
Equity Peak [$]                                 3.54404
Return [%]                                      216.438
Buy & Hold Return [%]                           184.485
Max. Drawdown [%]                              -47.2887
Avg. Drawdown [%]                              -4.85425
Max. Drawdown Duration                181 days 06:00:00
Avg. Drawdown Duration                  7 days 00:00:00
# Trades                                            357
Win Rate [%]                                     27.451
Best Trade [%]                                  31.9498
Worst Trade [%]                                -7.16953
Avg. Trade [%]                                 0.390037
Max. Trade Duration                    11 days 0

In [10]:
from backtesting import Strategy

def Breakout_Signaling(data, range_n, k):
    df = pd.DataFrame({'Open': data.Open, 'High': data.High, 'Low': data.Low, 'Close': data.Close})
    df['diff'] = df['High'] - df['Low']
    df['range'] = df['diff'].rolling(range_n).sum().fillna(df['diff'].rolling(range_n).sum().median())
    df['range_mean'] = df['range']/range_n
    df.loc[df['range_mean']*k+df['Open'] < df['High'], 'signal'] = 1.0
    df.loc[df['Open']-df['range_mean']*k > df['Low'], 'signal'] = -1.0
    return df['signal']

class ChannelBreakStrategy(Strategy):
    range_n = 5
    k = 1.6
    
    def init(self):
        self.signal = self.I(Breakout_Signaling, self.data, self.range_n, self.k)
        signal = pd.Series(self.signal)
        print('Signal count: {}'.format(signal.value_counts(ascending=True)))
    
    def next(self):
        if self.signal == 1.0:
            self.position.close()
            self.buy()
        elif self.signal == -1.0:
            self.position.close()
            self.sell()

In [11]:
from backtesting import Backtest
# start from 1 BTC with the commission 0.12%

df_ = df.copy()
bt_cb = Backtest(df_, ChannelBreakStrategy, cash=1, commission=.0012)
bt_cb.run()

Signal count: -1.0    593
 1.0    604
dtype: int64


Start                      2019-01-01 00:00:00
End                        2020-07-30 09:00:00
Duration                     576 days 09:00:00
Exposure [%]                           99.7614
Equity Final [$]                       1.38482
Equity Peak [$]                        2.11338
Return [%]                             38.4821
Buy & Hold Return [%]                  184.485
Max. Drawdown [%]                     -47.9293
Avg. Drawdown [%]                     -8.28654
Max. Drawdown Duration       216 days 12:00:00
Avg. Drawdown Duration        19 days 04:00:00
# Trades                                  1195
Win Rate [%]                           47.2803
Best Trade [%]                          19.849
Worst Trade [%]                       -12.8089
Avg. Trade [%]                       0.0466483
Max. Trade Duration            4 days 23:00:00
Avg. Trade Duration            0 days 12:00:00
Expectancy [%]                          1.5854
SQN                                   0.375296
Sharpe Ratio 

In [12]:
bt_cb.plot()